In [1]:
from IPython.display import display, HTML
display(HTML("<style>div.jp-OutputArea-output pre {white-space: pre;}</style>"))

from data_science.dao.utils.data_access import DaoContext
from data_science.spark.helpers import current_session

spark_opts = {
    "spark.dynamicAllocation.minExecutors": 6,
    "spark.executor.memory": "4G",
    "spark.executor.memoryOverhead": "8G",
    "spark.driver.cores": 6,
    "spark.driver.memory": "2G",
    "spark.driver.memoryOverhead": "8G",
    "spark.driver.maxResultSize": "16G"
}

context = DaoContext(**spark_opts)
context.__enter__()

current_session()

import pyspark.sql.functions as f
from pyspark.sql.functions import *
from pyspark.sql import Window
from pyspark.sql.types import *
from pyspark.sql.functions import udf
import pandas as pd
import random
import numpy as np

In [2]:
wz=(current_session().read.parquet("/vdp/vdp-rel1prd/datalake/raw/accepted/sas/prd/sasserver/StrategieMonitoring/wzw_beroep_l25m") )#.filter(f.col("observatieperiode").isin({'2022-05'}))

In [3]:
tbz=(current_session().read.parquet("/vdp/vdp-rel1prd/datalake/raw/accepted/sas/prd/sasserver/StrategieMonitoring/tbwz_dvl") )#.filter(f.col("observatieperiode").isin({'2022-05'}))

In [4]:
wz.count()

12860643

In [5]:
tbz.count()

11625015

In [6]:
tbz1=tbz.select ('klnr','observatieperiode','status_dvl_detail').withColumnRenamed("observatieperiode", "observatieperiode1").withColumnRenamed("klnr", "klnr1")

In [7]:
error_df = wz.join(tbz1,on=[wz.klnr==tbz1.klnr1,wz.observatieperiode==tbz1.observatieperiode1],how='left').drop('klnr1','observatieperiode1')

In [8]:
error_df.count()

Py4JJavaError: An error occurred while calling o140.count.
: org.apache.spark.util.SparkFatalException
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.$anonfun$relationFuture$1(BroadcastExchangeExec.scala:183)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withThreadLocalCaptured$1(SQLExecution.scala:185)
	at java.util.concurrent.FutureTask.run(FutureTask.java:266)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)


Op deze manier lukt de join niet, wel met de code hieronder, is dit de goede manier ? of ander voorstel?


In [9]:
from functools import reduce
list=wz.select(f.collect_set('observatieperiode').alias('observatieperiode')).first()['observatieperiode']
df_list = []
for x in list:
    wz1 = wz.filter("observatieperiode='%s'" % x)
    tbz2 = tbz1.filter("observatieperiode='%s'" % x)
    df_final = wz1.join(tbz2,on=[wz1.klnr==tbz2.klnr1,wz1.observatieperiode==tbz2.observatieperiode1],how='left').drop('klnr1','observatieperiode1')
    df_list.append(df_final)
    wz1.unpersist
    tbz2.unpersist
    df_final.unpersist
wz2 = reduce(lambda x, y: x.union(y), df_list)

In [10]:
wz2=wz2 .withColumn("soort_team",f.when(
                  ((f.col("team_bemiddelaar").isin({'Partner (GTB)','Servicelijn Generiek','Partner (Anderen)','Partner (OCMW)','VDAB (Anderen)'}))&(f.col("cluster_3D").isNotNull())),f.concat(f.col("team_bemiddelaar"),lit('_'), f.col("cluster_3D"))
                  ).when((f.col("team_bemiddelaar").isNull()),"Geen" 
                  ).when(((f.col("team_bemiddelaar").isin({'Partner (GTB)','Servicelijn Generiek','Partner (Anderen)','Partner (OCMW)','VDAB (Anderen)'}))&(f.col("cluster_3D").isNull())),f.col("team_bemiddelaar")
                  ).when(((f.col("team_bemiddelaar")!= 'Partner (GTB)')&(f.col("team_bemiddelaar")!= 'Servicelijn Generiek')&(f.col("team_bemiddelaar")!= 'Partner (Anderen)')&(f.col("team_bemiddelaar")!= 'Partner (OCMW)')&(f.col("team_bemiddelaar")!= 'VDAB (Anderen)')&(f.col("cluster_3D").isNull())),"VDAB_TEAM"
                  ).otherwise(f.concat(lit('VDAB_TEAM'),lit('_'), f.col("cluster_3D"))))

In [11]:
wz2.show()

+----------+-----------------------+--------------------+-----------------+--------------+---------------+-------------+---------------------+--------------------+--------------------+------+--------+-------+-------+--------------------+--------------------+------------------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+-----------------------+------------------------+------------------------------+------------------+--------------------+-------------------------+--------------------+---------------+--------------------+------------------+--------------------+--------------------+-----------------+-----------------+-----------------+--------------------+--------------------+-----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+---

In [12]:
wz2.count() 

12860643

In [13]:
def add_suffix(sdf, suffix):

      for c in sdf.columns:

            sdf = sdf.withColumnRenamed(c, '{}{}'.format(c, suffix))

      return sdf

In [14]:
wz2=add_suffix(wz2,'_wz')

In [15]:
wz2.show()

+----------+--------------------------+--------------------+--------------------+-----------------+------------------+----------------+------------------------+--------------------+--------------------+---------+-----------+----------+----------+--------------------+--------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------------+---------------------------+---------------------------------+---------------------+----------------------+----------------------------+--------------------+---------------+----------------------+------------------+--------------------+--------------------+-----------------+-----------------+-----------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+---------------------+--------------------+----------------------+---

In [16]:
vac=(current_session().read.parquet("/vdp/vdp-rel1prd/datalake/raw/accepted/sas/prd/sasserver/StrategieMonitoring/vacatures_detail") )

In [17]:

vac=vac.filter(f.col("maand").isin(list))

In [18]:
vac=add_suffix(vac,'_vac')

In [19]:
vac.show()

+---------------+------------------+--------------------+--------------------+-------------------+-------------------+-----------+--------------------+------------+---------------+---------+--------------------+--------------------------+-----------------------------+-----------------+------------+-----------------------+-----------------------+----------------------------+--------------------+-------+--------+--------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+------------------------+--------------------+--------------------+------------+------------+------------+--------------------+---------------+------------+---------------------+--------+-------------------+----------------------------+--------------------+-------------------+--------------------+-----------------------+----------------------------+-----------------+-----------------+----------------+--------------+--------------------+-----------------

In [20]:
vac.count()

4784117

In [21]:
wz2.columns

['periode_wz',
 'refregio_verblijfplaats_wz',
 'sv_verblijfplaats_wz',
 'observatieperiode_wz',
 'observatiejaar_wz',
 'observatiemaand_wz',
 'inzetbaarheid_wz',
 'provincie_bemiddelaar_wz',
 'BESCHIKBAARHEID_wz',
 'STRAAT_wz',
 'HUISNR_wz',
 'GESLACHT_wz',
 'STUDIES_wz',
 'NISCODE_wz',
 'SMP_ONDERSTEUNING_wz',
 'INZETBAARHEID_OMS_wz',
 'streektafel_wz',
 'gemeentecluster_wz',
 'post_gemeente_wz',
 'wzw_wz',
 'catwz_wz',
 'catwz_detail_wz',
 'leeftijd_jaar_wz',
 'leeftijd_basis_wz',
 'leeftijd_detail_wz',
 'werkloosheidsduur_basis_wz',
 'werkloosheidsduur_detail_wz',
 'werkloosheidsduur_detail_langd_wz',
 'studieniveau_basis_wz',
 'studieniveau_detail_wz',
 'studieniveau_detail_extra_wz',
 'studierichting_wz',
 'studiegebied_wz',
 'studierichting_code_wz',
 'arrondissement_wz',
 'oe_verblijfplaats_wz',
 'regio_wz',
 'fusiegemeente_wz',
 'provincie_wz',
 'zorggebied_wz',
 'resoc_wz',
 'serr_wz',
 'nationaliteitland_wz',
 'nationaliteit_3l_wz',
 'nationaliteit_8l_wz',
 'werksituatie_wz',

In [22]:
wz3=wz2.select(
    
 'observatieperiode_wz',
 'inzetbaarheid_wz',
 'provincie_bemiddelaar_wz',
 'BESCHIKBAARHEID_wz',
 'GESLACHT_wz',
 'STUDIES_wz',
 'NISCODE_wz',
 'SMP_ONDERSTEUNING_wz',
 'INZETBAARHEID_OMS_wz',
 'wzw_wz',
 'catwz_wz',
 'catwz_detail_wz',
 'leeftijd_jaar_wz',
 'leeftijd_basis_wz',
 'leeftijd_detail_wz',
 'werkloosheidsduur_basis_wz',
 'werkloosheidsduur_detail_wz',
 'werkloosheidsduur_detail_langd_wz',
 'studieniveau_basis_wz',
 'studieniveau_detail_wz',
 'studieniveau_detail_extra_wz',
 'studierichting_wz',
 'studiegebied_wz',
 'studierichting_code_wz',
 'nationaliteitland_wz',
 'nationaliteit_3l_wz',
 'nationaliteit_8l_wz',
 'werksituatie_wz',
 'bemiddelingsstatus_wz',
 'team_bemiddelaar_wz',
 'cluster_bemiddelaar_wz',
 'cluster_3D_wz',
 'provincie_bemiddelaar_1l_wz',
 'bemiddelaar_naam_wz',
 'klnr_wz',
 'arbeidsbeperking_wz',
 'arbeidshandicap_wz',
 'origine_wz',
 'migratieachtergrond_wz',
 'land_xp__wz',
 'origineland_wz',
 'origine_3l_wz',
 'origine_8l_wz',
 'kennis_nederlands_wz',
 'kennis_engels_wz',
 'kennis_frans_wz', 
 'talrijkheid_beroepen_wz',
 'kleur_kansopwerk_wz',
 'beroep_wz',
 'regime_zelfstandig_wz',
 'rijbewijs_wz',
 'rijbewijs_B_wz',
 'rijbewijs_C_wz',
 'rijbewijs_D_wz', 
 'datum_wz',
 'ervaring_wz',
 'status_dvl_detail_wz',
 'soort_team_wz'
)

In [23]:
wz3.count()

12860643

In [24]:
vac.columns

['maand_tekst_vac',
 'periode_12mndn_vac',
 'periode_vac',
 'rijbewijs_ja_nee_vac',
 'rijbewijs_soort_vac',
 'circuit_vac',
 'niscode_vac',
 'studieniveau_vac',
 'ervaring_vac',
 'titel_datum_vac',
 'maand_vac',
 'beroepp_oms_vac',
 'beroepencluster_comeet_vac',
 'beroepensubcluster_comeet_vac',
 'arbeidsregime_vac',
 'tijd_vac',
 'beroepenveld_comeet_vac',
 'beroepsgroep_comeet_vac',
 'hoofdberoepsgroep_comeet_vac',
 'studieniv_detail_vac',
 'DIM_vac',
 'jaar_vac',
 'beheer_vac',
 'cirgroep_oms_vac',
 'circuit_oms_vac',
 'provincie_vestiging_vac',
 'resoc_vac',
 'referentieregio_vac',
 'samenwerkingsverband_vac',
 'SECTOR_WSE_vac',
 'SECTOR_ARV_vac',
 'knel2020_vac',
 'knel2021_vac',
 'knel2022_vac',
 'provincie_beheer_vac',
 'sjabloon_id_vac',
 'BEMIDDEL_vac',
 'vestiging_bedrijf_vac',
 'stem_vac',
 'postnr_gemeente_vac',
 'arrondissement_vestiging_vac',
 'regio_vestiging_vac',
 'gemeentecluster_vac',
 'team_bemiddelaar_vac',
 'cluster_bemiddelaar_vac',
 'cluster_bemiddelaar_code_vac

In [25]:
vac1=vac.select(
     
 'periode_12mndn_vac',
 'periode_vac',
 'rijbewijs_ja_nee_vac',
 'rijbewijs_soort_vac',
 'circuit_vac',
 'niscode_vac',
 'studieniveau_vac',
 'ervaring_vac',
 'maand_vac',
 'beroepp_oms_vac',
 'arbeidsregime_vac',
 'tijd_vac',
 'studieniv_detail_vac',
 'DIM_vac',
 'beheer_vac',
 'cirgroep_oms_vac',
 'circuit_oms_vac',
 'samenwerkingsverband_vac',
 'SECTOR_WSE_vac',
 'SECTOR_ARV_vac',
 'provincie_beheer_vac',
 'BEMIDDEL_vac',
 'team_bemiddelaar_vac',
 'cluster_bemiddelaar_vac',
 'taalniv_N_om_vac',
 'taalniv_F_om_vac',
 'taalniv_E_om_vac',
 'taalniv_D_om_vac',
 'NL_vraag_vac',
 'FR_vraag_vac',
 'EN_vraag_vac',
 'DU_vraag_vac',
 'aantalhm_vac',
 'vervulhm_vac',
 'annnvvhm_vac',
 'saldo_vac',
 'taal_nl_vac',
 'taal_fr_vac',
 'taal_en_vac',
 'taal_du_vac',
 'taal_an_vac',
 'geen_nl_vac',
 'blanco_vac',
 'geen_rijverdeling_vac',
 'geen_categorische_onderverdeling_vac',
 'totaal_vac',
 'totaal2_vac',
 'ontvangen_25_tot_36_m_vac',
 'ontvangen_13_tot_24_m_vac',
 'ontvangen_laatste12maanden_vac',
 
)   

In [26]:
wz3.show()

+--------------------+----------------+------------------------+--------------------+-----------+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------------+---------------------------+---------------------------------+---------------------+----------------------+----------------------------+--------------------+---------------+----------------------+--------------------+-------------------+-------------------+--------------------+---------------------+--------------------+----------------------+--------------------+---------------------------+--------------------+----------+-------------------+------------------+----------+----------------------+-----------+--------------+-------------+-------------+--------------------+------------------+------------------+-----------------------+-------------------+--------------------+-------------

In [27]:
vac1.show()

+------------------+--------------------+--------------------+-------------------+-------------------+-----------+--------------------+------------+---------+--------------------+-----------------+------------+--------------------+-------+--------------------+--------------------+--------------------+------------------------+--------------------+--------------------+--------------------+------------+--------------------+-----------------------+--------------------+--------------------+--------------------+--------------------+------------+------------+------------+------------+------------+------------+------------+---------+-----------+-----------+-----------+-----------+-----------+-----------+----------+---------------------+------------------------------------+----------+-----------+-------------------------+-------------------------+------------------------------+
|periode_12mndn_vac|         periode_vac|rijbewijs_ja_nee_vac|rijbewijs_soort_vac|        circuit_vac|niscode_vac|    

In [28]:
wz3=wz3.na.fill(value='Geen', subset=['provincie_bemiddelaar_wz'])

In [29]:
wz3=wz3.na.replace(
    ['Geen','Regionale dienst Brussel','Provincie Limburg','Provincie Vlaams-Brabant','Provincie West-Vlaanderen','Centraal Bestuur VDAB','Provincie Antwerpen','Provincie Oost-Vlaanderen'],
    ['Geen','Brussel','Limburg','Vlaams-Brabant','West-Vlaanderen','Centrale Dienst','Antwerpen','Oost-Vlaanderen'], 
    'provincie_bemiddelaar_wz')

In [30]:
wz4=(wz3
     .withColumnRenamed("observatieperiode_wz","Maand")
     .withColumnRenamed("NISCODE_wz","Niscode")
     .withColumnRenamed("beroep_wz","Beroep")
     .withColumnRenamed("provincie_bemiddelaar_wz","VDAB_PROV")
     .withColumnRenamed("cluster_bemiddelaar_wz","cluster_bemiddelaar")
)

In [31]:
vac2=(vac1
     .withColumnRenamed("maand_vac","Maand")
     .withColumnRenamed("niscode_vac","Niscode")
     .withColumnRenamed("beroepp_oms_vac","Beroep")
     .withColumnRenamed("provincie_beheer_vac","VDAB_PROV")
     .withColumnRenamed("cluster_bemiddelaar_vac","cluster_bemiddelaar")
)

In [32]:
vac2.printSchema()

root
 |-- periode_12mndn_vac: string (nullable = true)
 |-- periode_vac: string (nullable = true)
 |-- rijbewijs_ja_nee_vac: string (nullable = true)
 |-- rijbewijs_soort_vac: string (nullable = true)
 |-- circuit_vac: string (nullable = true)
 |-- Niscode: string (nullable = true)
 |-- studieniveau_vac: string (nullable = true)
 |-- ervaring_vac: string (nullable = true)
 |-- Maand: string (nullable = true)
 |-- Beroep: string (nullable = true)
 |-- arbeidsregime_vac: string (nullable = true)
 |-- tijd_vac: string (nullable = true)
 |-- studieniv_detail_vac: string (nullable = true)
 |-- DIM_vac: string (nullable = true)
 |-- beheer_vac: string (nullable = true)
 |-- cirgroep_oms_vac: string (nullable = true)
 |-- circuit_oms_vac: string (nullable = true)
 |-- samenwerkingsverband_vac: string (nullable = true)
 |-- SECTOR_WSE_vac: string (nullable = true)
 |-- SECTOR_ARV_vac: string (nullable = true)
 |-- VDAB_PROV: string (nullable = true)
 |-- BEMIDDEL_vac: string (nullable = true)


In [33]:
wz4 = wz4.withColumn('leeftijd_jaar_wz', f.col('leeftijd_jaar_wz').cast(StringType()))

In [34]:
wz4.printSchema()

root
 |-- Maand: string (nullable = true)
 |-- inzetbaarheid_wz: string (nullable = true)
 |-- VDAB_PROV: string (nullable = false)
 |-- BESCHIKBAARHEID_wz: string (nullable = true)
 |-- GESLACHT_wz: string (nullable = true)
 |-- STUDIES_wz: string (nullable = true)
 |-- Niscode: string (nullable = true)
 |-- SMP_ONDERSTEUNING_wz: string (nullable = true)
 |-- INZETBAARHEID_OMS_wz: string (nullable = true)
 |-- wzw_wz: string (nullable = true)
 |-- catwz_wz: string (nullable = true)
 |-- catwz_detail_wz: string (nullable = true)
 |-- leeftijd_jaar_wz: string (nullable = true)
 |-- leeftijd_basis_wz: string (nullable = true)
 |-- leeftijd_detail_wz: string (nullable = true)
 |-- werkloosheidsduur_basis_wz: string (nullable = true)
 |-- werkloosheidsduur_detail_wz: string (nullable = true)
 |-- werkloosheidsduur_detail_langd_wz: string (nullable = true)
 |-- studieniveau_basis_wz: string (nullable = true)
 |-- studieniveau_detail_wz: string (nullable = true)
 |-- studieniveau_detail_extr

In [35]:
from pyspark.sql.functions import lit

for column in [column for column in vac2.columns if column not in wz4.columns]:
    wz4 = wz4.withColumn(column, lit('a'))


for column in [column for column in wz4.columns if column not in vac2.columns]:
    vac2 = vac2.withColumn(column, lit('a'))

In [36]:
wz4.show()

+-------+----------------+---------------+--------------------+-----------+----------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------------+---------------------------+---------------------------------+---------------------+----------------------+----------------------------+--------------------+---------------+----------------------+--------------------+-------------------+-------------------+--------------------+---------------------+--------------------+--------------------+--------------------+---------------------------+--------------------+----------+-------------------+------------------+----------+----------------------+-----------+--------------+-------------+-------------+--------------------+------------------+------------------+-----------------------+-------------------+--------------------+---------------------+------------+-----

In [37]:
wz4=(wz4
.na.replace( ['a'],['0.0'], 'aantalhm_vac')
.na.replace( ['a'],['0.0'], 'vervulhm_vac')
.na.replace( ['a'],['0.0'], 'annnvvhm_vac')
.na.replace( ['a'],['0.0'], 'saldo_vac')
.na.replace( ['a'],['0.0'], 'taal_nl_vac')
.na.replace( ['a'],['0.0'], 'taal_fr_vac')
.na.replace( ['a'],['0.0'], 'taal_en_vac')
.na.replace( ['a'],['0.0'], 'taal_du_vac')
.na.replace( ['a'],['0.0'], 'taal_an_vac')
.na.replace( ['a'],['0.0'], 'geen_nl_vac')
.na.replace( ['a'],['0.0'], 'ontvangen_25_tot_36_m_vac')
.na.replace( ['a'],['0.0'], 'ontvangen_13_tot_24_m_vac')
.na.replace( ['a'],['0.0'], 'ontvangen_laatste12maanden_vac')   
)  

In [38]:
wz4=(wz4
     .withColumn('aantalhm_vac', f.col('aantalhm_vac').cast(DoubleType()))
     .withColumn('vervulhm_vac', f.col('vervulhm_vac').cast(DoubleType()))
     .withColumn('annnvvhm_vac', f.col('annnvvhm_vac').cast(DoubleType()))
     .withColumn('saldo_vac', f.col('saldo_vac').cast(DoubleType()))
     .withColumn('taal_nl_vac', f.col('taal_nl_vac').cast(DoubleType()))
     .withColumn('taal_fr_vac', f.col('taal_fr_vac').cast(DoubleType()))
     .withColumn('taal_en_vac', f.col('taal_en_vac').cast(DoubleType()))
     .withColumn('taal_du_vac', f.col('taal_du_vac').cast(DoubleType()))
     .withColumn('taal_an_vac', f.col('taal_an_vac').cast(DoubleType()))
     .withColumn('geen_nl_vac', f.col('geen_nl_vac').cast(DoubleType()))
     .withColumn('ontvangen_25_tot_36_m_vac', f.col('ontvangen_25_tot_36_m_vac').cast(DoubleType()))
     .withColumn('ontvangen_13_tot_24_m_vac', f.col('ontvangen_13_tot_24_m_vac').cast(DoubleType()))
     .withColumn('ontvangen_laatste12maanden_vac', f.col('ontvangen_laatste12maanden_vac').cast(DoubleType()))
    )      

In [39]:
wz4.show()

+-------+----------------+---------------+--------------------+-----------+----------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------------+---------------------------+---------------------------------+---------------------+----------------------+----------------------------+--------------------+---------------+----------------------+--------------------+-------------------+-------------------+--------------------+---------------------+--------------------+--------------------+--------------------+---------------------------+--------------------+----------+-------------------+------------------+----------+----------------------+-----------+--------------+-------------+-------------+--------------------+------------------+------------------+-----------------------+-------------------+--------------------+---------------------+------------+-----

In [40]:
wz4.printSchema()

root
 |-- Maand: string (nullable = true)
 |-- inzetbaarheid_wz: string (nullable = true)
 |-- VDAB_PROV: string (nullable = false)
 |-- BESCHIKBAARHEID_wz: string (nullable = true)
 |-- GESLACHT_wz: string (nullable = true)
 |-- STUDIES_wz: string (nullable = true)
 |-- Niscode: string (nullable = true)
 |-- SMP_ONDERSTEUNING_wz: string (nullable = true)
 |-- INZETBAARHEID_OMS_wz: string (nullable = true)
 |-- wzw_wz: string (nullable = true)
 |-- catwz_wz: string (nullable = true)
 |-- catwz_detail_wz: string (nullable = true)
 |-- leeftijd_jaar_wz: string (nullable = true)
 |-- leeftijd_basis_wz: string (nullable = true)
 |-- leeftijd_detail_wz: string (nullable = true)
 |-- werkloosheidsduur_basis_wz: string (nullable = true)
 |-- werkloosheidsduur_detail_wz: string (nullable = true)
 |-- werkloosheidsduur_detail_langd_wz: string (nullable = true)
 |-- studieniveau_basis_wz: string (nullable = true)
 |-- studieniveau_detail_wz: string (nullable = true)
 |-- studieniveau_detail_extr

In [41]:
vac2.show()

+------------------+--------------------+--------------------+-------------------+-------------------+-------+--------------------+------------+-------+--------------------+-----------------+------------+--------------------+-------+--------------------+--------------------+--------------------+------------------------+--------------------+--------------------+---------------+------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+------------+------------+------------+------------+------------+------------+------------+---------+-----------+-----------+-----------+-----------+-----------+-----------+----------+---------------------+------------------------------------+----------+-----------+-------------------------+-------------------------+------------------------------+----------------+------------------+-----------+----------+--------------------+--------------------+------+--------+---------------+-

In [42]:
wzvac=wz4.unionByName(vac2)


In [43]:
wzvac.show()

+-------+----------------+---------------+--------------------+-----------+----------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------------+---------------------------+---------------------------------+---------------------+----------------------+----------------------------+--------------------+---------------+----------------------+--------------------+-------------------+-------------------+--------------------+---------------------+--------------------+--------------------+--------------------+---------------------------+--------------------+----------+-------------------+------------------+----------+----------------------+-----------+--------------+-------------+-------------+--------------------+------------------+------------------+-----------------------+-------------------+--------------------+---------------------+------------+-----

In [44]:
a = wzvac.tail(20)
print(a)

[Row(Maand='2022-03', inzetbaarheid_wz='a', VDAB_PROV='Centrale Dienst', BESCHIKBAARHEID_wz='a', GESLACHT_wz='a', STUDIES_wz='a', Niscode='71022', SMP_ONDERSTEUNING_wz='a', INZETBAARHEID_OMS_wz='a', wzw_wz='a', catwz_wz='a', catwz_detail_wz='a', leeftijd_jaar_wz='a', leeftijd_basis_wz='a', leeftijd_detail_wz='a', werkloosheidsduur_basis_wz='a', werkloosheidsduur_detail_wz='a', werkloosheidsduur_detail_langd_wz='a', studieniveau_basis_wz='a', studieniveau_detail_wz='a', studieniveau_detail_extra_wz='a', studierichting_wz='a', studiegebied_wz='a', studierichting_code_wz='a', nationaliteitland_wz='a', nationaliteit_3l_wz='a', nationaliteit_8l_wz='a', werksituatie_wz='a', bemiddelingsstatus_wz='a', team_bemiddelaar_wz='a', cluster_bemiddelaar=None, cluster_3D_wz='a', provincie_bemiddelaar_1l_wz='a', bemiddelaar_naam_wz='a', klnr_wz='a', arbeidsbeperking_wz='a', arbeidshandicap_wz='a', origine_wz='a', migratieachtergrond_wz='a', land_xp__wz='a', origineland_wz='a', origine_3l_wz='a', origin

In [45]:
wzvac.count()

17644760

In [46]:
12860643+4784117

17644760

In [47]:
b=(current_session().read.parquet("/vdp/vdp-rel1prd/datalake/raw/accepted/sas/prd/sasserver/StrategiePlanning/comeetberoepenformat") )
c=(current_session().read.parquet("/vdp/vdp-rel1prd/datalake/raw/accepted/sas/prd/sasserver/StrategiePlanning/niscodeformat") )

In [48]:
b.show()

+-----------+--------+------------+--------------------+-----------------+--------------------+--------------+--------------------+-------------------+--------------------+--------------+--------+--------------------+----+------+--------------+--------+--------+--------+------+--------+--------+--------+--------+--------+--------+--------+------------------+--------------------+
|sjabloon_id|fiche_id|beroepencode|     beroepencluster|beroepencluster_p|        beroepenveld|beroepenveld_p|   hoofdberoepsgroep|hoofdberoepsgroep_p|        beroepsgroep|beroepsgroep_p|beroep_p|              beroep|stem|status|knelpuntberoep|knel2015|knel2016|knel2017|isco08|knel2018|knel2019|knel2020|knel2013|knel2014|knel2021|knel2022|beroepensubcluster|beroepensubcluster_p|
+-----------+--------+------------+--------------------+-----------------+--------------------+--------------+--------------------+-------------------+--------------------+--------------+--------+--------------------+----+------+-------

In [49]:
b.count()

681

In [50]:
b=b.filter(f.col("status").isin({'A'}))

In [51]:
b=b.drop('beroep','status')

In [52]:
b.count()

643

In [53]:
c.count()

608

In [54]:
c=c.filter(f.col("status").isin({'A'}))

In [55]:
c=c.drop('status')

In [56]:
c.count()

571

In [57]:
c.show()

+-------+-----------+----------+----------+----------+-------+---------+----------------+----------------+-----------+-----------+--------+----------+-------+---------+------+---------+-------------+-------------+------------+------------+-----------+------------+------------+-----------------+-----------------+----------------------+----------------------+----------+---------+
|niscode|niscode2019|  gemeente|gemeente_p|gemeente_l|regio_p|  regio_l|arrondissement_p|arrondissement_l|provincie_p|provincie_l|gewest_p|  gewest_l|resoc_p|  resoc_l|serr_p|   serr_l|streektafel_p|streektafel_l|zorggebied_p|zorggebied_l|centrumstad|gemcluster_p|gemcluster_l|referentieregio_p|referentieregio_l|samenwerkingsverband_p|samenwerkingsverband_l| gewest2_l|gewest2_p|
+-------+-----------+----------+----------+----------+-------+---------+----------------+----------------+-----------+-----------+--------+----------+-------+---------+------+---------+-------------+-------------+------------+------------

In [58]:
wzvac1=wzvac.withColumn('beroep_p1', split(wzvac['Beroep'], ' ').getItem(0))

In [59]:
wzvac1.show()

+-------+----------------+---------------+--------------------+-----------+----------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------------+---------------------------+---------------------------------+---------------------+----------------------+----------------------------+--------------------+---------------+----------------------+--------------------+-------------------+-------------------+--------------------+---------------------+--------------------+--------------------+--------------------+---------------------------+--------------------+----------+-------------------+------------------+----------+----------------------+-----------+--------------+-------------+-------------+--------------------+------------------+------------------+-----------------------+-------------------+--------------------+---------------------+------------+-----

In [60]:
wzvac2=wzvac1.join(c,on=[wzvac1.Niscode==c.niscode],how='left').drop('niscode')

In [61]:
wzvac2.show()

+-------+----------------+---------------+--------------------+-----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------------+---------------------------+---------------------------------+---------------------+----------------------+----------------------------+--------------------+---------------+----------------------+--------------------+-------------------+-------------------+--------------------+---------------------+--------------------+--------------------+--------------------+---------------------------+--------------------+----------+-------------------+------------------+----------+----------------------+-----------+--------------+-------------+-------------+--------------------+------------------+------------------+-----------------------+-------------------+--------------------+---------------------+------------+-------------

In [62]:
wzvac2.count()

17644760

In [63]:
wzvac3=wzvac2.join(b,on=[wzvac2.beroep_p1==b.beroep_p],how='left').drop('beroep_p1')

In [64]:
wzvac3.show()

+-------+----------------+---------------+--------------------+-----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------------+---------------------------+---------------------------------+---------------------+----------------------+----------------------------+--------------------+---------------+----------------------+--------------------+-------------------+-------------------+--------------------+---------------------+--------------------+--------------------+--------------------+---------------------------+--------------------+----------+-------------------+------------------+----------+----------------------+-----------+--------------+-------------+-------------+--------------------+------------------+------------------+-----------------------+-------------------+--------------------+---------------------+------------+-------------

In [65]:
wzvac3.count()

17644760

In [66]:
b.groupBy('beroep_p').count().orderBy('count', ascending=False).show()


+--------+-----+
|beroep_p|count|
+--------+-----+
|  BA3210|    1|
|  CE1040|    1|
|  VA1410|    1|
|  ID3990|    1|
|  IN1720|    1|
|  IK5820|    1|
|  CB1220|    1|
|  HB3910|    1|
|  IB1410|    1|
|  ID3822|    1|
|  CB7210|    1|
|  BB5820|    1|
|  VB7820|    1|
|  HC1221|    1|
|  VA1820|    1|
|  LA3991|    1|
|  AA1231|    1|
|  AD3141|    1|
|  MA1230|    1|
|  IG7810|    1|
+--------+-----+
only showing top 20 rows



In [67]:
wzvac3.columns

['Maand',
 'inzetbaarheid_wz',
 'VDAB_PROV',
 'BESCHIKBAARHEID_wz',
 'GESLACHT_wz',
 'STUDIES_wz',
 'SMP_ONDERSTEUNING_wz',
 'INZETBAARHEID_OMS_wz',
 'wzw_wz',
 'catwz_wz',
 'catwz_detail_wz',
 'leeftijd_jaar_wz',
 'leeftijd_basis_wz',
 'leeftijd_detail_wz',
 'werkloosheidsduur_basis_wz',
 'werkloosheidsduur_detail_wz',
 'werkloosheidsduur_detail_langd_wz',
 'studieniveau_basis_wz',
 'studieniveau_detail_wz',
 'studieniveau_detail_extra_wz',
 'studierichting_wz',
 'studiegebied_wz',
 'studierichting_code_wz',
 'nationaliteitland_wz',
 'nationaliteit_3l_wz',
 'nationaliteit_8l_wz',
 'werksituatie_wz',
 'bemiddelingsstatus_wz',
 'team_bemiddelaar_wz',
 'cluster_bemiddelaar',
 'cluster_3D_wz',
 'provincie_bemiddelaar_1l_wz',
 'bemiddelaar_naam_wz',
 'klnr_wz',
 'arbeidsbeperking_wz',
 'arbeidshandicap_wz',
 'origine_wz',
 'migratieachtergrond_wz',
 'land_xp__wz',
 'origineland_wz',
 'origine_3l_wz',
 'origine_8l_wz',
 'kennis_nederlands_wz',
 'kennis_engels_wz',
 'kennis_frans_wz',
 'talr

In [68]:

wzvac3.write.partitionBy("Maand").mode("overwrite").parquet("sandbox/centrale_diensten/arbeidsmarktbeheer/strategie/planning/common/mg/test/MergeWzVac")


code loop zoder error maar bestand niet in Dremio" sandbox/centrale_diensten/arbeidsmarktbeheer/strategie/planning/common/mg/test" weggeschreven

In [85]:
context.__exit__()